# Projet 5 :  Segmentez des clients d'un site e-commerce
<font color = "grey"><i> Chargement des modules necessaires </i></font>

## Updated featuring



In [848]:

import pandas as pd

In [849]:
import plotly.graph_objects as go
import plotly.express as px
from sklearn import preprocessing
from sklearn.cluster import KMeans

In [850]:
full_df = pd.read_pickle('data/complete_df')
full_df.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value,total_order_payment_amount,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,quarter,recency
0,b81ef226f3fe1789b1e8b2acac839d17,1.0,credit_card,8.0,99.33,99.33,0a8556ac6be836b46b3e89920d59291c,708ab75d2a007f0564aedd11139c7708,39801,teofilo otoni,MG,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,2018-05-09 17:36:51,2018-05-22,2018Q2,470.0
1,a9810da82917af2d9aefd1278f1dcfa0,1.0,credit_card,1.0,24.39,24.39,f2c7fc58a9de810828715166c672f10a,a8b9d3a27068454b1c98cc67d4e31e6f,2422,sao paulo,SP,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,2018-06-29 20:32:09,2018-07-16,2018Q2,228.0
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1.0,credit_card,1.0,65.71,65.71,25b14b69de0b6e184ae6fe2755e478f9,6f70c0b2f7552832ba46eb57b1c5651e,2652,sao paulo,SP,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04,2017Q4,102.0
3,ba78997921bbcdc1373bb41e913ab953,1.0,credit_card,8.0,107.78,107.78,7a5d8efaaa1081f800628c30d2b0728f,87695ed086ebd36f20404c82d20fca87,36060,juiz de fora,MG,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04,2017Q4,169.0
4,42fdf880ba16b47b59251dd489d4441a,1.0,credit_card,2.0,128.45,128.45,15fd6fb8f8312dbb4674e4518d6fa3b3,4291db0da71914754618cd789aebcd56,18570,conchas,SP,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,2018-06-01 21:44:53,2018-06-13,2018Q2,31.0


In [851]:
full_rfm = pd.read_pickle('data/new_rfm_df')
full_rfm.columns

Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'total_order_payment_amount',
       'customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'quarter', 'recency', 'late_delivery',
       'product_id', 'seller_id', 'freight_value', 'seller_city',
       'seller_state', 'frequency', 'monetary', 'product_category_name',
       'same_state'],
      dtype='object')

In [852]:
full_rfmc = pd.read_pickle('data/rfm_complete')
full_rfmc.head()


,Récence,Frequence,Montant
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,123,1,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,126,1,27.19
0000f46a3911fa3c0805444483337064,548,1,86.22
0000f6ccb0745a6a4b88665a16c9f078,333,1,43.62
0004aac84e0df4da2b147fca70cf8255,299,1,196.89


In [853]:
full_rfmc.rename(columns={'Récence': 'recency',
                          'Montant':'monetary',
                          'Frequence':'frequency'}, inplace=True)


In [854]:
full_rfmc.head()

,recency,frequency,monetary
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,123,1,141.90
0000b849f77a49e4a4ce2b2a4ca5be3f,126,1,27.19
0000f46a3911fa3c0805444483337064,548,1,86.22
0000f6ccb0745a6a4b88665a16c9f078,333,1,43.62
0004aac84e0df4da2b147fca70cf8255,299,1,196.89


In [855]:
full_rfmc = full_rfmc.join(full_rfm[['customer_unique_id','freight_value', 'payment_installments']]
         .groupby('customer_unique_id').min(),on='customer_unique_id')

In [856]:
full_rfmc.head()

,recency,frequency,monetary,freight_value,payment_installments
customer_unique_id,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,123,1,141.90,12.00,8.0
0000b849f77a49e4a4ce2b2a4ca5be3f,126,1,27.19,8.29,1.0
0000f46a3911fa3c0805444483337064,548,1,86.22,17.22,8.0
0000f6ccb0745a6a4b88665a16c9f078,333,1,43.62,17.63,4.0
0004aac84e0df4da2b147fca70cf8255,299,1,196.89,16.89,6.0


In [857]:
full_rfmc.to_pickle('data/rfm_df_best')

In [858]:
full_df["order_purchase_timestamp"] = pd.to_datetime(full_df['order_purchase_timestamp'])
#
full_df["order_purchase_year"] =  pd.to_datetime(full_df['order_purchase_timestamp']).dt.year
full_df["order_purchase_month"] = pd.to_datetime(full_df['order_purchase_timestamp']).dt.month

full_df_2017 = full_df.loc[full_df['order_purchase_year'] == 2017]
full_df_2017_tri_1  = full_df.loc[(full_df['order_purchase_year'] == 2017) & (full_df['order_purchase_month'] < 4 )]
full_df_2017_tri_2  = full_df.loc[(full_df['order_purchase_year'] == 2017) & (full_df['order_purchase_month'] < 3 ) & (full_df['order_purchase_month'] < 7 )]
full_df_2017_sem_1  = full_df.loc[(full_df['order_purchase_year'] == 2017) & (full_df['order_purchase_month'] < 7 )]
full_df_2017_sem_2  = full_df.loc[(full_df['order_purchase_year'] == 2017) & (full_df['order_purchase_month'] > 6 )]


In [859]:
full_df_2017_sem_2.head()


,order_id,payment_sequential,payment_type,payment_installments,payment_value,total_order_payment_amount,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,...,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,quarter,recency,order_purchase_year,order_purchase_month
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1.0,credit_card,1.0,65.71,65.71,25b14b69de0b6e184ae6fe2755e478f9,6f70c0b2f7552832ba46eb57b1c5651e,2652,sao paulo,...,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04,2017Q4,102.0,2017,12
3,ba78997921bbcdc1373bb41e913ab953,1.0,credit_card,8.0,107.78,107.78,7a5d8efaaa1081f800628c30d2b0728f,87695ed086ebd36f20404c82d20fca87,36060,juiz de fora,...,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04,2017Q4,169.0,2017,12
9,0573b5e23cbd798006520e1d5b4c6714,1.0,boleto,1.0,51.95,51.95,c4e17eff78176dfe3401f03db0346f85,511c345b0383af6ef25406c564e4d6b6,17780,lucelia,...,delivered,2017-07-04 20:32:10,2017-07-06 03:15:36,2017-07-06 13:27:31,2017-07-13 19:34:27,2017-07-26,2017Q3,233.0,2017,7
10,d88e0d5fa41661ce03cf6cf336527646,1.0,credit_card,8.0,188.73,188.73,fe0c5425832ac53c3be5cb4aea25b60f,a51f8568095ce89a4ffce365dcb7d46d,17501,marilia,...,delivered,2017-11-28 15:53:29,2017-11-28 16:17:30,2017-12-12 23:22:02,2017-12-19 20:49:52,2017-12-18,2017Q4,275.0,2017,11
11,2480f727e869fdeb397244a21b721b67,1.0,credit_card,1.0,141.90,141.90,94967f1e8a9ea8ec6c7129f098f32155,2faf4a1d502b10555a9f39353ea20148,13607,araras,...,delivered,2017-12-13 18:51:45,2017-12-13 18:59:27,2017-12-14 17:45:03,2017-12-19 18:33:08,2018-01-11,2017Q4,203.0,2017,12


In [860]:
def cluster(data,RFM):
    RFM = RFM.dropna()
    RFM = RFM.drop(RFM.index.difference(data['customer_unique_id']))

    X = RFM[['recency', 'monetary', 'frequency','freight_value','payment_installments']].values
    std_scale = preprocessing.StandardScaler().fit(X)
    X_scaled = std_scale.transform(X)

    model = KMeans(n_clusters=8, init='random', n_init=10)
    b = model.fit(X_scaled).labels_

    RFM['Segment'] = model.predict(X_scaled)+1

    B = RFM.groupby('Segment').mean()[
        ['recency', 'monetary', 'frequency','freight_value','payment_installments']]


    # Normalisation des données de RFM_clu entre **0 et 1000** :
    X_clu = B[['recency', 'monetary', 'frequency','freight_value','payment_installments']].values
    MinMax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1000))\
                                .fit(X_clu)
    X_clu_scaled = MinMax_scale.transform(X_clu)
    B_scale = pd.DataFrame(X_clu_scaled, columns=['recency',
                                                        'monetary',
                                                        'frequency','freight_value','payment_installments'])


    categories = ['Récence moyenne', 'Frequence moyenne', 'Montant moyen','freight_value','payment_installments']
    inc = 1
    for i in range(0,3):
        fig = go.Figure()

        for j in range(0,3):
            fig.add_trace(go.Scatterpolar(
              r=[B_scale['recency'].iloc[inc-1],
                 B_scale['frequency'].iloc[inc-1],
                 B_scale['monetary'].iloc[inc-1],
                 B_scale['freight_value'].iloc[inc-1],
                 B_scale['payment_installments'].iloc[inc-1]],
                 theta=categories,
                 fill='toself',
                 name=f'Cluster {inc}  '
                 ))


            if inc == 8:
                break
            inc = inc + 1
        fig.update_layout(
        polar=dict(
        radialaxis=dict(
          visible=False,
          range=[0, 1000]
        )),
        title="Radar Chart des Clusters ",
        showlegend=True
        )
        fig.show()

# 2017

In [861]:
cluster(full_df_2017,full_rfmc)

# 2017 1er semestre

In [862]:
cluster(full_df_2017_sem_1,full_rfmc)

# 2017 2e semestre

In [863]:
cluster(full_df_2017_sem_2,full_rfmc)

# 2017 1e trimestre

In [864]:
cluster(full_df_2017_tri_1,full_rfmc)

# 2017 2e trimestre

In [865]:
cluster(full_df_2017_tri_2,full_rfmc)

> Pour le contrat de mise à jour des clusters je vais rester sur une periode de 3 mois de mise a jours.
Pour avoir un résultat sur les nouveaux client plus précis et avoir les meilleurs featuring

In [866]:
product_perception = pd.read_pickle('data/rfm_cluster_customer_id')
product_perception = product_perception.dropna()

In [867]:
product_perception = product_perception.join(full_rfm[['customer_unique_id','product_category_name']]
         .groupby('customer_unique_id').min(),on='customer_unique_id')

In [868]:
#recency max and frequency min count produit
#
product_perception.head()

,Récence,Frequence,Montant,Cluster Labels,Segment,product_category_name
customer_unique_id,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,123,1,141.90,7,Cluster 8,cama_mesa_banho
0000b849f77a49e4a4ce2b2a4ca5be3f,126,1,27.19,6,Cluster 7,beleza_saude
0000f46a3911fa3c0805444483337064,548,1,86.22,3,Cluster 4,papelaria
0000f6ccb0745a6a4b88665a16c9f078,333,1,43.62,1,Cluster 2,telefonia
0004aac84e0df4da2b147fca70cf8255,299,1,196.89,2,Cluster 3,telefonia


> Le cluster 1 Corresponds au client loyal par la fréquence max s'y trouve
> Le cluster 4 Corresponds au nouveau  client car la frequence est min et la récence est max
> Le cluster 5 Corresponds au client dépensié

In [869]:
df_loyal = product_perception[product_perception['Cluster Labels']== 0]
df_new = product_perception[product_perception['Cluster Labels']== 3]
df_rich = product_perception[product_perception['Cluster Labels']== 4]

In [870]:
product_perception['Segment'] = product_perception['Segment'].astype('category')
product_perception['Segment'] = product_perception['Segment']\
                        .cat.reorder_categories(['Cluster 1',
                                                 'Cluster 2',
                                                 'Cluster 3',
                                                 'Cluster 4',
                                                 'Cluster 5',
                                                 'Cluster 6',
                                                 'Cluster 7',
                                                 'Cluster 8'])
product_perception

,Récence,Frequence,Montant,Cluster Labels,Segment,product_category_name
customer_unique_id,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,123,1,141.90,7,Cluster 8,cama_mesa_banho
0000b849f77a49e4a4ce2b2a4ca5be3f,126,1,27.19,6,Cluster 7,beleza_saude
0000f46a3911fa3c0805444483337064,548,1,86.22,3,Cluster 4,papelaria
0000f6ccb0745a6a4b88665a16c9f078,333,1,43.62,1,Cluster 2,telefonia
0004aac84e0df4da2b147fca70cf8255,299,1,196.89,2,Cluster 3,telefonia
...,...,...,...,...,...,...
fffbf87b7a1a6fa8b03f081c5f51a201,250,1,167.32,2,Cluster 3,NaN
fffea47cd6d3cc0a88bd621562a9d061,265,1,84.58,1,Cluster 2,bebes
ffff371b4d645b6ecea244b27531430a,581,1,112.46,3,Cluster 4,automotivo


In [871]:
RFM_clu = product_perception.groupby('Segment')\
                     .agg(Récence_mean=('Récence', 'mean'),
                          Frequence_mean=('Frequence', 'mean'),
                          Montant_mean=('Montant', 'mean'),
                          Total=('Récence', 'count'))
RFM_clu

,Récence_mean,Frequence_mean,Montant_mean,Total
Segment,,,,
Cluster 1,252.138179,2.186985,165.339191,4733
Cluster 2,263.292358,1.000000,65.816549,18686
Cluster 3,293.280525,1.000000,163.556179,10598
Cluster 4,474.406651,1.000000,81.647302,14194
Cluster 5,183.064977,1.000000,333.546164,5248
Cluster 6,466.441088,1.000000,264.300302,4006
Cluster 7,95.914882,1.000000,69.819912,19009
Cluster 8,108.856397,1.000000,175.119299,11685


In [872]:
df_loyal = df_loyal.dropna()
df_new = df_new.dropna()
df_rich = df_rich.dropna()

# Produit les plus consommés par les clients loyal

In [873]:
df_loyal_count = pd.DataFrame({'product_category_name':df_loyal['product_category_name'].value_counts().head(10).index,
              'nombre':df_loyal['product_category_name'].value_counts().head(10).values})
df_loyal_count

,product_category_name,nombre
0,cama_mesa_banho,709
1,beleza_saude,429
2,esporte_lazer,411
3,moveis_decoracao,318
4,informatica_acessorios,282
5,utilidades_domesticas,219
6,brinquedos,202
7,automotivo,188
8,relogios_presentes,177
9,ferramentas_jardim,171


In [874]:
fig_1 = px.histogram(df_loyal_count,
                     title="Produit les plus acheter par les client fidèle",
                     x="product_category_name",
                     y='nombre',

                     color="product_category_name",
                     color_discrete_sequence=px.colors.qualitative.Pastel2,
                     barmode="group"

                     ).update_xaxes(categoryorder="total ascending")

fig_1.show()

# Produit les plus consommés par les nouveau clients

In [875]:
df_new_count = pd.DataFrame({'product_category_name':df_new['product_category_name'].value_counts().head(10).index,
              'nombre':df_new['product_category_name'].value_counts().head(10).values})
df_new_count

,product_category_name,nombre
0,cama_mesa_banho,1337
1,moveis_decoracao,1177
2,esporte_lazer,1147
3,beleza_saude,1120
4,utilidades_domesticas,993
5,informatica_acessorios,901
6,telefonia,850
7,brinquedos,747
8,cool_stuff,729
9,ferramentas_jardim,539


In [876]:
fig_1 = px.histogram(df_new_count,
                     title="Produit les plus acheter par les nouveaux clients ",
                     x="product_category_name",
                     y='nombre',

                     color="product_category_name",
                     color_discrete_sequence=px.colors.qualitative.Pastel2,
                     barmode="group"

                     ).update_xaxes(categoryorder="total ascending")

fig_1.show()

# Produit les plus consommés par les dépensiés

In [877]:
df_rich_count = pd.DataFrame({'product_category_name':df_rich['product_category_name'].value_counts().head(10).index,
              'nombre':df_rich['product_category_name'].value_counts().head(10).values})
df_rich_count

,product_category_name,nombre
0,beleza_saude,608
1,esporte_lazer,421
2,relogios_presentes,415
3,moveis_decoracao,397
4,cama_mesa_banho,329
5,informatica_acessorios,280
6,utilidades_domesticas,277
7,cool_stuff,237
8,automotivo,231
9,perfumaria,228


In [878]:
fig_1 = px.histogram(df_rich_count,
                     title="Produit les plus acheter par les clients dépensiés",
                     x="product_category_name",
                     y='nombre',
                     color="product_category_name",
                     color_discrete_sequence=px.colors.qualitative.Pastel2,
                     barmode="group"
                     ).update_xaxes(categoryorder="total ascending")

fig_1.show()
